In [1]:
import pandas as pd
import os

os.chdir("/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_MeCP2_CUTandTAG/Methylation_dev")

In [2]:
experiment = "align2_005"
# Load the merged results
df = pd.read_csv(f"analyze_mecp2_cpg_enrichment_{experiment}/NSC_methylation_analysis.csv")
df.head()

,gene_id,gene_name,expression_status,log2FoldChange,padj,promoter_methylation,promoter_cpg_count,gene_body_methylation,gene_body_cpg_count,mecp2_bound,binding_type,exo_signal,endo_signal
0,ENSMUSG00000050530,Fam171a1,unchanged,0.086201,3.309688e-01,83.333333,22,100.000000,1271,True,exo_only,170.045154,0.000000
1,ENSMUSG00000026643,Nmt2,unchanged,-0.125745,9.951615e-02,46.660744,130,100.000000,305,True,both,1334.759216,1586.675997
2,ENSMUSG00000026648,Dclre1c,downregulated,-0.570326,3.277770e-07,83.333025,51,100.000000,354,True,both,224.682107,532.384150
3,ENSMUSG00000085043,Gm13184,downregulated,-0.902754,2.509670e-03,100.000000,14,99.880719,67,True,both,66.823411,206.598678
4,ENSMUSG00000026655,Fam107b,downregulated,-0.826802,1.140920e-15,100.000000,17,100.000000,1518,True,both,146.960535,21.869481


In [4]:
# Analyze each binding category
categories = {
    'exo_enriched': df[(df['binding_type'] == 'exo') | 
                                ((df['binding_type'] == 'both') & 
                                 (df['exo_signal'] > df['endo_signal']))],
    'endo_only': df[df['binding_type'] == 'endo'],
    'non_enriched': df[(df['binding_type'] == 'both') & 
                                (df['exo_signal'] <= df['endo_signal'])]
}

for category, data in categories.items():
    print(f"\n{category.upper()} Analysis:")
    # Count regulation status
    print(data['expression_status'].value_counts())
    # Check methylation levels
    print("\nMethylation levels by regulation status:")
    print(data.groupby('expression_status')[['promoter_methylation', 
                                           'gene_body_methylation']].mean())


EXO_ENRICHED Analysis:
expression_status
unchanged        396
upregulated      104
downregulated     93
Name: count, dtype: int64

Methylation levels by regulation status:
                   promoter_methylation  gene_body_methylation
expression_status                                             
downregulated                 88.796245              92.595300
unchanged                     83.584271              92.844760
upregulated                   79.995086              88.976186

ENDO_ONLY Analysis:
Series([], Name: count, dtype: int64)

Methylation levels by regulation status:
Empty DataFrame
Columns: [promoter_methylation, gene_body_methylation]
Index: []

NON_ENRICHED Analysis:
expression_status
unchanged        83
downregulated    27
upregulated      21
Name: count, dtype: int64

Methylation levels by regulation status:
                   promoter_methylation  gene_body_methylation
expression_status                                             
downregulated                 84.5